# Instally Snowflake and making the connection

In [ ]:
pip install pyarrow==10.0.1

In [2]:
pip install --upgrade snowflake-connector-python

  Using cached snowflake_connector_python-3.12.1-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (64 kB)
  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl.metadata (13 kB)
  Using cached tomlkit-0.13.2-py3-none-any.whl.metadata (2.7 kB)
Using cached snowflake_connector_python-3.12.1-cp311-cp311-manylinux_2_17_aarch64.manylinux2014_aarch64.whl (2.5 MB)
Using cached asn1crypto-1.5.1-py2.py3-none-any.whl (105 kB)
Using cached tomlkit-0.13.2-py3-none-any.whl (37 kB)
Note: you may need to restart the kernel to use updated packages.


In [58]:
import snowflake.connector

In [59]:
conn = snowflake.connector.connect(
    user='SIDDHARTHAMYSORE',
    password='Fall@2024',
    account='luqhchh-xnb07739',
    )

In [60]:
cs = conn.cursor()

Creating a Warehouse, database, schema and Stage

In [7]:
cs.execute("CREATE WAREHOUSE IF NOT EXISTS ETL_Group_2_9")

In [8]:
cs.execute("CREATE DATABASE IF NOT EXISTS ELT_Project_Database")

In [9]:
cs.execute("CREATE SCHEMA IF NOT EXISTS CSV_Schema")

In [10]:
cs.execute("CREATE OR REPLACE STAGE CSV_Stage")

# Extract and load the 41 comma delimited purchases data files
and form a single table of purchases data; a. Preferably follow these guidelines when staging the files (this staging approach does not make sense for our data as the files are small, but it is good practice if you have more data and if the data is loaded over time) b. Use Python to automate the PUT process, e.g., use glob to iterate through and PUT all purchases files automatically c. COPY INTO is generally preferred over INSERT INTO (this applies to the entire project); d. To the extent possible, perform transformations such as selecting columns and setting data types during the COPY INTO process

Creating a table called "MonthlyPurchaseOrders" to load the 41 csv files

In [11]:
cs.execute(
    """
    CREATE OR REPLACE TABLE MonthlyPurchaseOrders (
        PurchaseOrderID INT,
        SupplierID INT,
        OrderDate DATE,
        DeliveryMethodID INT,
        ContactPersonID INT,
        ExpectedDeliveryDate DATE,
        SupplierReference VARCHAR(255),
        IsOrderFinalized BOOLEAN,
        Comments VARCHAR,
        InternalComments VARCHAR,
        LastEditedBy INT,
        LastEditedWhen TIMESTAMP,
        PurchaseOrderLineID INT,
        StockItemID INT,
        OrderedOuters INT,
        Description VARCHAR,
        ReceivedOuters INT,
        PackageTypeID INT,
        ExpectedUnitPricePerOuter FLOAT,
        LastReceiptDate DATE,
        IsOrderLineFinalized BOOLEAN,
        Right_LastEditedBy INT,
        Right_LastEditedWhen TIMESTAMP
    )
    """
)


# Importing Glob to load all the csv files at once

In [12]:
import glob

In [13]:
files = glob.glob("/home/jovyan/etl/Monthly_PO_Data/*.csv")

Staging the files to the stage called "CSV_Stage"

In [14]:
for file_path in files:
    put_command = f"PUT file://{file_path} @CSV_stage"
    cs.execute(put_command)

# Copying the csv files to the MonthlyPurchaseOrder table

In [16]:
copy_command = """
COPY INTO MonthlyPurchaseOrders
FROM @CSV_stage
FILE_FORMAT = (
    TYPE = 'CSV'
    FIELD_OPTIONALLY_ENCLOSED_BY='"'
    SKIP_HEADER=1
    TIMESTAMP_FORMAT = 'MM/DD/YYYY HH:MI'
    DATE_FORMAT = 'MM/DD/YYYY'
    NULL_IF = ('NULL', '00:00.0', '2/29/2022', '2/28/2022 7:00:00 AM')
)
ON_ERROR = 'CONTINUE';
"""
cs.execute(copy_command)
print("All staged files have been loaded into 'MonthlyPurchaseOrders'.")

All staged files have been loaded into 'MonthlyPurchaseOrders'.


In [17]:
sql = """
ALTER TABLE MonthlyPurchaseOrders
ADD COLUMN POAmount DECIMAL(18, 2);
"""
cs.execute(sql)

# 2. Creating a calculated field that shows purchase order totals,
i.e., for each order, sum the line item amounts (defined as ReceivedOuters * ExpectedUnitPricePerOuter), and name this field POAmount



In [18]:
sql = """
UPDATE MonthlyPurchaseOrders
SET POAmount = (
    SELECT SUM(p.ReceivedOuters * p.ExpectedUnitPricePerOuter)
    FROM MonthlyPurchaseOrders p
    WHERE p.PurchaseOrderID = MonthlyPurchaseOrders.PurchaseOrderID
);
"""
cs.execute(sql)

# 3. Extract and Load Supplier invoice data
shred the data into a table (preferably in the COPY INTO process) where each row corresponds to a single invoicevoice

In [34]:
cs.execute("CREATE OR REPLACE STAGE XML_Stage")

In [36]:
import os
file_path = os.getcwd() + "/SupplierTransactionsXML.xml"
stage_name = "XML_Stage"
put_command = f"PUT file://{file_path} @{stage_name}"
cs.execute(put_command)

In [42]:
create_table_command = """CREATE OR REPLACE TABLE SupplierTransactions(xml_data VARIANT NOT NULL);"""
cs.execute(create_table_command)

In [43]:
copy= f""" COPY INTO SupplierTransactions FROM @XML_stage FILE_FORMAT = (TYPE = 'XML') ON_ERROR='CONTINUE';"""
cs.execute(copy)

In [44]:
cs.execute(""" 
CREATE OR REPLACE TABLE SupplierTransactions AS
SELECT
    XMLGET( r.value, 'SupplierTransactionID' ):"$"::INTEGER AS SupplierTransactionID,
    XMLGET( r.value, 'SupplierID' ):"$"::INTEGER AS SupplierID,
    XMLGET( r.value, 'TransactionTypeID' ):"$"::INTEGER AS TransactionTypeID,
    CASE WHEN REGEXP_LIKE(XMLGET(r.value, 'PurchaseOrderID'):"$", '^[0-9]+$') THEN TO_NUMBER(XMLGET(r.value, 'PurchaseOrderID'):"$") ELSE NULL END AS PurchaseOrderID,
    XMLGET( r.value, 'PaymentMethodID' ):"$"::INTEGER AS PaymentMethodID,
    CASE WHEN REGEXP_LIKE(XMLGET(r.value, 'SupplierInvoiceNumber'):"$", '^[0-9]+$') THEN TO_NUMBER(XMLGET(r.value, 'SupplierInvoiceNumber'):"$") ELSE NULL END AS SupplierInvoiceNumber,
    XMLGET( r.value, 'TransactionDate' ):"$"::STRING AS TransactionDate,
    XMLGET( r.value, 'AmountExcludingTax' ):"$"::FLOAT AS AmountExcludingTax,
    XMLGET( r.value, 'TaxAmount' ):"$"::FLOAT AS TaxAmount,
    XMLGET( r.value, 'TransactionAmount' ):"$"::FLOAT AS TransactionAmount,
    XMLGET( r.value, 'OutstandingBalance' ):"$"::FLOAT AS OutstandingBalance,
    XMLGET( r.value, 'FinalizationDate' ):"$"::STRING AS FinalizationDate,
    XMLGET( r.value, 'IsFinalized' ):"$"::INTEGER AS IsFinalized,
    XMLGET( r.value, 'LastEditedBy' ):"$"::INTEGER AS LastEditedBy,
    XMLGET( r.value, 'LastEditedWhen' ):"$"::STRING AS LastEditedWhen
FROM SupplierTransactions,  
LATERAL FLATTEN(SupplierTransactions.xml_data:"$") r
WHERE GET( r.value, '@') = 'row' """)

rows = cs.fetchall()
for row in rows:
    print(row)

('Table SUPPLIERTRANSACTIONS successfully created.',)


# 4. Join the purchases data from step 2 and the supplier invoices data
(only include matching rows); assuming that step 2 was completed correctly, you can assume the following relationships among the four tables (the other two tables are discussed below):

In [47]:
command = """
CREATE OR REPLACE TABLE Supplier_Purchases AS
SELECT a.Purchaseorderid, a.supplierid, a.orderdate, a.EXPECTEDDELIVERYDATE, a.ISORDERLINEFINALIZED, a.PURCHASEORDERLINEID, a.STOCKITEMID, 
a.ORDEREDOUTERS, a.RECEIVEDOUTERS, a.EXPECTEDUNITPRICEPEROUTER, a.LASTRECEIPTDATE, a.POAMOUNT, b.SUPPLIERTRANSACTIONID, b.SUPPLIERINVOICENUMBER, 
b.TRANSACTIONDATE, b.AMOUNTEXCLUDINGTAX, b.TAXAMOUNT, TRANSACTIONAMOUNT, b.FINALIZATIONDATE
FROM MonthlyPurchaseOrders a
INNER JOIN SupplierTransactions b on a.supplierid = b.supplierid and a.PURCHASEORDERID = b.PURCHASEORDERID
"""
cs.execute(command)

# 5. Create a calculated field that shows the difference between AmountExcludingTax and POAmount
Name this field invoiced_vs_quoted, and save the result as a materialized view named purchase_orders_and_invoices

In [87]:
command = """
CREATE OR REPLACE VIEW PURCHASE_ORDERS_AND_INVOICES AS (
SELECT AMOUNTEXCLUDINGTAX, POAMOUNT, (AMOUNTEXCLUDINGTAX - POAMOUNT) AS invoiced_vs_quoted
FROM Supplier_Purchases
)
"""
cs.execute(command)

In [89]:
cs.execute("select * from PURCHASE_ORDERS_AND_INVOICES order by invoiced_vs_quoted desc limit 10")
rows = cs.fetchmany(10)
for row in rows:
    print(rows)


[(1171782.0, Decimal('410124.00'), 761658.0), (1171782.0, Decimal('410124.00'), 761658.0), (1171782.0, Decimal('410124.00'), 761658.0), (1171782.0, Decimal('410124.00'), 761658.0), (1171782.0, Decimal('410124.00'), 761658.0), (1171782.0, Decimal('410124.00'), 761658.0), (1118178.0, Decimal('391332.00'), 726846.0), (1118178.0, Decimal('391332.00'), 726846.0), (1118178.0, Decimal('391332.00'), 726846.0), (1118178.0, Decimal('391332.00'), 726846.0)]
[(1171782.0, Decimal('410124.00'), 761658.0), (1171782.0, Decimal('410124.00'), 761658.0), (1171782.0, Decimal('410124.00'), 761658.0), (1171782.0, Decimal('410124.00'), 761658.0), (1171782.0, Decimal('410124.00'), 761658.0), (1171782.0, Decimal('410124.00'), 761658.0), (1118178.0, Decimal('391332.00'), 726846.0), (1118178.0, Decimal('391332.00'), 726846.0), (1118178.0, Decimal('391332.00'), 726846.0), (1118178.0, Decimal('391332.00'), 726846.0)]
[(1171782.0, Decimal('410124.00'), 761658.0), (1171782.0, Decimal('410124.00'), 761658.0), (117178

# 6. Extract the supplier_case data from postgres
- Consider creating a Python function that can take a csv file path as input and then generate field definitions (field names and datatypes based on the header and data types in the file) that can then be used in CREATE TABLE statement.

- You need to use psycopg2 or a similar Python library to connect to the postgres database within Python, issue a command to postgres to have postgres save the supplier_case data to file, and then use cs.execute to move the file to an internal Snowflake stage and eventually into a table.

In [53]:
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="WestCoastImporters",
    user="jovyan",  # The existing superuser
    password="postgres",  # The password for the postgres user
    host="127.0.0.1",  # Or the IP address of your PostgreSQL server
    port="8765" )

In [55]:
cs = conn.cursor()
directory = os.getcwd()

# Check if the directory exists, if not, create it
if not os.path.exists(directory):
    os.makedirs(directory)

# Now define the full path for the CSV file
csv_file_path = os.path.join(directory, 'supplier_case.csv')

query = "COPY (SELECT * FROM supplier_case) TO STDOUT WITH CSV HEADER"
with open(csv_file_path, 'w') as f:
    cs.copy_expert(query, f)

In [62]:
cs.execute("Use database ELT_Project_Database ")


In [66]:
cs.execute("CREATE OR REPLACE STAGE Csv_Stage_Supplier_Base")

In [65]:
cs.execute("USE SCHEMA CSV_SCHEMA")

In [68]:
csv_file = os.getcwd() + "/supplier_case.csv"
cs.execute(f"PUT file://{csv_file} @Csv_Stage_Supplier_Base")

In [73]:
command = """
CREATE OR REPLACE FILE FORMAT SUPPLIER_CASE 
    TYPE = CSV,
    FIELD_OPTIONALLY_ENCLOSED_BY = '"',
    PARSE_HEADER = TRUE,
    TIMESTAMP_FORMAT = 'MM/DD/YYYY HH24:MM',
"""
cs.execute(command)

In [80]:
command = """
    CREATE TABLE supplier_case (
    supplierid INT,
    suppliername VARCHAR(255),
    suppliercategoryid INT,
    primarycontactpersonid INT,
    alternatecontactpersonid INT,
    deliverymethodid INT,
    postalcityid INT,
    supplierreference VARCHAR(50),
    bankaccountname VARCHAR(255),
    bankaccountbranch VARCHAR(255),
    bankaccountcode VARCHAR(50),
    bankaccountnumber VARCHAR(50),
    bankinternationalcode VARCHAR(50),
    paymentdays INT,
    internalcomments TEXT,
    phonenumber VARCHAR(50),
    faxnumber VARCHAR(50),
    websiteurl VARCHAR(255),
    deliveryaddressline1 VARCHAR(255),
    deliveryaddressline2 VARCHAR(255),
    deliverypostalcode VARCHAR(20),
    deliverylocation VARCHAR(255),
    postaladdressline1 VARCHAR(255),
    postaladdressline2 VARCHAR(255),
    postalpostalcode VARCHAR(20),
    lasteditedby INT,
    validfrom VARCHAR(50),
    validto VARCHAR(50)
);
"""
cs.execute(command)

In [81]:
command = """
    COPY INTO supplier_case FROM @Csv_Stage_Supplier_Base
        FILE_FORMAT = SUPPLIER_CASE
        MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
    ON_ERROR = 'CONTINUE'
"""
cs.execute(command)

# 7. Connect manually to NOAA data using Marketplace and work on the weather data

In [97]:
cs.execute("USE DATABASE ELT_PROJECT_DATABASE")

In [98]:
cs.execute("USE SCHEMA CSV_SCHEMA")

In [99]:
cs.execute("CREATE STAGE zip_codes")

In [101]:
file = os.getcwd() + "/2021_Gaz_zcta_national.txt"

cs.execute(f"PUT file://{file} @zip_codes")

In [107]:
cs.execute("""CREATE OR REPLACE TABLE zcta_national (
    GEOID VARCHAR(5),
    ALAND NUMERIC,
    AWATER NUMERIC,
    ALAND_SQMI FLOAT,
    AWATER_SQMI FLOAT,
    INTPTLAT FLOAT,
    INTPTLONG FLOAT
);
""")

In [113]:
command = """
COPY INTO zcta_national
FROM @zip_codes
FILE_FORMAT = (
    TYPE = 'CSV',
    FIELD_OPTIONALLY_ENCLOSED_BY = '"',
    FIELD_DELIMITER = '\t',
    SKIP_HEADER = 1
)
"""
cs.execute(command)

In [116]:
command = """
    CREATE OR REPLACE TABLE SUPPLIER_ZIP AS (
        SELECT a.postalpostalcode, b.INTPTLAT AS LATITUDE, b.INTPTLONG AS LONGITUDE
        FROM supplier_case a
        JOIN zcta_national b ON a.postalpostalcode = b.GEOID
        GROUP BY a.postalpostalcode, b.INTPTLAT, b.INTPTLONG);
"""

cs.execute(command)

In [121]:
command = """
CREATE OR REPLACE TABLE zip_weather AS
    SELECT 
        a.postalpostalcode,
        b.noaa_weather_station_id AS StationID,
        b.noaa_weather_station_name AS StationName,
        SQRT(
            POW(69.1 * (a.latitude - b.latitude), 2) +
            POW(69.1 * (b.longitude - a.longitude) * COS(a.latitude / 57.3), 2)
        ) AS distance
    FROM ELT_PROJECT_DATABASE.CSV_SCHEMA.SUPPLIER_ZIP a
    CROSS JOIN WEATHER__ENVIRONMENT.cybersyn.noaa_weather_station_index b
    QUALIFY ROW_NUMBER() OVER (PARTITION BY a.postalpostalcode ORDER BY distance ASC) = 1;
"""

cs.execute(command)

In [123]:
command = """
CREATE OR REPLACE TABLE zip_weather_data AS
           SELECT 
            a.postalpostalcode,
            b.date,
            b.value as Max_Temp,
            b.variable
            FROM ELT_PROJECT_DATABASE.CSV_SCHEMA.zip_weather a
            JOIN WEATHER__ENVIRONMENT.cybersyn.noaa_weather_metrics_timeseries b
            ON a.STATIONID = b.noaa_weather_station_id
            WHERE b.variable = 'maximum_temperature';
"""
cs.execute(command)

In [125]:
command = """
CREATE OR REPLACE TABLE zip_weather_station_mapping_temp AS
    SELECT 
        a.postalpostalcode,
        b.noaa_weather_station_id AS station_id,
        b.noaa_weather_station_name AS station_name,
        st_distance(
            st_makepoint(a.longitude, b.latitude),
            st_makepoint(b.longitude, a.latitude)
        ) AS distance
    FROM ELT_PROJECT_DATABASE.CSV_SCHEMA.SUPPLIER_ZIP a
    JOIN WEATHER__ENVIRONMENT.cybersyn.noaa_weather_station_index b
    ON st_distance(st_makepoint(a.longitude, a.latitude), st_makepoint(b.longitude, b.latitude)) < 50000
    QUALIFY ROW_NUMBER() OVER (PARTITION BY a.postalpostalcode ORDER BY distance ASC) = 1;
"""

cs.execute(command)

In [126]:
command = """
    CREATE OR REPLACE TABLE zip_code_weather_data_temp AS
           SELECT 
            a.postalpostalcode,
            b.date,
            b.value as Max_Temp,
            b.variable
            FROM ELT_PROJECT_DATABASE.CSV_SCHEMA.zip_weather_station_mapping_temp a
            JOIN WEATHER__ENVIRONMENT.cybersyn.noaa_weather_metrics_timeseries b
            ON a.station_id = b.noaa_weather_station_id
            WHERE b.variable = 'maximum_temperature';
"""

cs.execute(command)

In [127]:
command = """
CREATE OR REPLACE TABLE supplier_zip_code_weather_temp2 AS 
SELECT a.PostalPostalCode, b.date, b.Max_temp FROM supplier_case as a
JOIN zip_code_weather_data_temp as b ON a.postalpostalcode = b.postalpostalcode
ORDER BY DATE;
"""

cs.execute(command)

In [129]:
command = """
CREATE OR REPLACE TABLE supplier_zip_code_weather_temp AS 
SELECT A.PostalPostalCode, B.date, B.Max_temp FROM supplier_case as A
LEFT JOIN zip_weather_data as B ON A.postalpostalcode = B.postalpostalcode
ORDER BY DATE;
"""
cs.execute(command)


In [130]:
command = """CREATE OR REPLACE MATERIALIZED VIEW supplier_zip_code_weather AS (SELECT * FROM supplier_zip_code_weather_temp);"""

cs.execute(command)

# 8. Join purchase_orders_and_invoices, supplier_case, and supplier_zip_code_weather
Based on zip 
codes and the transaction date. Only include transactions that have matching temperature readings.

In [134]:
cs.execute("""
CREATE OR REPLACE VIEW order_zip_temp AS 
SELECT 
    a.*, 
    b.SUPPLIERID AS SupplierID1, 
    c.SUPPLIERID AS SupplierID2
FROM 
    supplier_zip_code_weather AS a
LEFT JOIN 
    supplier_purchases AS b 
    ON a.date = b.OrderDate  
LEFT JOIN 
    supplier_case AS C 
    ON a.postalpostalcode = c.postalpostalcode  
WHERE 
    a.max_temp IS NOT NULL;
""")

cs.execute(command)
